In [ ]:
#import all packages
import pandas as pd
import numpy as np
import re as re
import nltk
from nltk.stem.porter import *

from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier

from datetime import datetime, timedelta

In [ ]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [ ]:
# datafile
data_file = "../LazadaData/Lazada_AllProducts_160220.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#reformat the respective columns

#lower case
data['Review'] = [word.lower() for word in data['Review']]

#Strip off the Variation Word
data['Product_Purchase'] = data['Product_Purchase'].str.strip("Variation:")

#Change Date of Review to DateTime
#data['Date_Of_Review'] = pd.to_datetime(data['Date_Of_Review'])

data.info()
data.head()

### Transformation and Initalise of data

In [ ]:
# make the old csv data into a list
row_id_list = data['Unnamed:_0'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()

# initialising the new columns
id_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_csv = []
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

In [ ]:
#upload the appos
from word_dict import appos

print(appos)

### Transform the Date Format [Optional]

In [2]:
new_date =[]
for i in range(len(brand_list)):
    
    date_review = date_review_list[i]
    
    if platform_list[i] == 'Lazada':
        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = datetime.today() - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = datetime.today() - timedelta(days=day)
            date_review = date_review.strftime("%d-%b-%y")
#             print("trans",date_review)

        elif 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = datetime.today() - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%b-%y")
    
#         print("trans",date_review)
        new_date.append(date_review)
    else:
#         print(platform_list[i])
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

NameError: name 'brand_list' is not defined

### Spilt the Sentence 

In [ ]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    #print(current_review)
    
    #spilt the sentence
    splitted_sentence= re.split(r'[.!]', current_review) 
    print(splitted_sentence)
    
    for j in range(len(splitted_sentence)):
        
        splitted_sentence[j].strip()
        if len(splitted_sentence[j]) > 0 :
            
            processed_review = splitted_sentence[j].strip()
            
            # Negation handling
            processed_review = processed_review.split()
            processed_review =[appos[w] if w in appos else w for w in processed_review]
            processed_review = " ".join(processed_review) 
            
            processed_review = processed_review.strip()
            # Remove all the special characters
            processed_review = re.sub(r'\W', ' ', processed_review)
            # Removing prefixed 'b'
            processed_review = re.sub(r'^b\s+', '', processed_review)
            # remove all single characters contains a white space character
            processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
            # Substituting multiple spaces with single space
            processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
          
            
            processed = processed_review.strip()
            print(processed)
            
            # final check if the processed_review is empty
            final = []
            if processed != "":
                tokenized = word_tokenize(processed)
                #removed additional letters eg "gooooooodddddd" to "good"
                for w in tokenized:
                    processed_2 = reduce_lengthening(w)
                    processed_2 = spell.correction(processed_2)
                    final.append(processed_2)
                final =' '.join(final)
                print(final)
                # append into a dictionary
                id_csv.append(row_id_list[i])
                brand_csv.append(brand_list[i])
                category_csv.append(category_list[i])
                product_name_csv.append(product_name_list[i])
                prices_csv.append(price_list[i])
                reviewer_csv.append(reviewer_list[i])
                review_csv.append(final)
                product_variation_csv.append(product_variation_list[i])
                rating_csv.append(rating_list[i])
                date_review_csv.append(new_date_review_list[i])
                response_csv.append(response_list[i])

            
#Store into a new data frame
splitted_data = {'ID':id_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv, 'Product Purchase':product_variation_csv,'Ratings':rating_csv,'Date Of Review':date_review_csv,'Response': response_csv }
splitted_df = pd.DataFrame.from_dict(splitted_data)
# splitted_df.to_csv('Lazada_process_v1.csv')
splitted_df.head()
splitted_df.info()

### Create the Training Data Set

In [ ]:
# spacy for lemmatization
import spacy

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader

review_cleaned=[]
stop_list = stopwords.words('english')

stemmer = PorterStemmer()
sp = spacy.load('en', disable=['parser', 'ner'])

splitted_df= splitted_df[splitted_df.Review != 'no comments review is an image']
tokenized_review = splitted_df["Review"]


for sentence in tokenized_review:
    tokenized_review = nltk.word_tokenize(sentence)
    review_stopremoved = [w for w in tokenized_review if w not in stop_list]
    #print(review_stopremoved)
    review_cleaned.append(review_stopremoved)
    

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(review_cleaned, allowed_postags=['NOUN', 'ADJ','VERB'])

## Find a way to update the review 

splitted_df['tokenized_review'] = data_lemmatized    
splitted_df.head()

### Import the Positive and Negative Words


In [ ]:
import nltk 

pos_lexicon = 'dict/positive-words.txt'
neg_lexicon = 'dict/negative-words.txt'
inc_words = 'dict/increment-words.txt'
dec_words = 'dict/decrement-words.txt'
inv_words = 'dict/inverse-words.txt'


# Read the positive sentiment lexicon.
pos_dict = {}
f = open(pos_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    pos_dict[line] = 1
f.close()

# Read the negative sentiment lexicon.
neg_dict = {}
f = open(neg_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    neg_dict[line] = 1
f.close()


# Read the increment words.
inc_dict = {}
f = open(inc_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inc_dict[line] = 1
f.close()

# Read the decrement words.
dec_dict = {}
f = open(dec_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    dec_dict[line] = 1
f.close()

# Read the inverse words.
inv_dict = {}
f = open(inv_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inv_dict[line] = 1
f.close()


In [ ]:
# import nltk
# from nltk.tokenize import word_tokenize

# review_comment_text = "extremely poor service"

# review_1 = word_tokenize(review_comment_text) 
# # review = [nltk.word_tokenize(w)for w in review_comment_text]
# #review = [pos_tag(w) for w in review]

# # tagsText= nltk.pos_tag(review)
# print(review_1)

In [ ]:
# review_clean = [review_1]

# for sent in review_clean:
#     total_score = 0
#     for w in sent:
#         print(w)
#         score = 0
#         # If the word w is inside the positive lexicon, then increase the score by 1.
#         if w in pos_dict:
#             score = 1
#         elif w in neg_dict:
#             score = -1
#         position = sent.index(w)
#         previous_word = sent[position-1]
#         if previous_word is not None:
#             if previous_word in inc_dict : 
#                 print(score)
#                 score *= 2.0
#             elif previous_word in dec_dict:
#                 print(score)
#                 score /=2
#             elif previous_word in inv_dict:
#                 score *= -1  
#         total_score+=score
#         print(score)
#         print(total_score)
#     print(total_score)

In [ ]:
predicted_labels = []
score_label=[]

for sent in data_lemmatized:
    total_score = 0
    print(sent)
    for w in sent:
        score = 0
        if w in pos_dict:
            score = 1
        elif w in neg_dict:
            score = -1
        #check previous word
        position = sent.index(w)
        previous_word = sent[position-1]
        if previous_word is not None:
            if previous_word in inc_dict : 
                score *= 2.0
            elif previous_word in dec_dict:
                score /=2
            elif previous_word in inv_dict:
                score *= -1  
        total_score+=score
    
    if total_score >0:
        predicted_labels.append('1')
        
    elif total_score <0:
        predicted_labels.append('-1')
    else:
        predicted_labels.append('0')
        
    score_label.append(int(total_score))
    print(total_score)

splitted_df["Polarity"] = predicted_labels
splitted_df["Score"] = score_label
splitted_df.to_csv('Lazada_sentiment.csv')
